<a href="https://colab.research.google.com/github/AFFANAHMED-17/MLExperiments/blob/main/MLEXP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, cross_validate
ratings_path = "/content/ratings.csv"
movies_path = "/content/movies.csv"
ratings_df = pd.read_csv(ratings_path)
movies_df = pd.read_csv(movies_path)
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)
svd_model = SVD()
svd_model.fit(trainset)
cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
def recommend_movies(user_id, model, movies_df, n=10):
    all_movie_ids = movies_df['movieId'].unique()
    user_rated_movies = ratings_df[ratings_df['userId'] == user_id]['movieId']
    unrated_movie_ids = [m for m in all_movie_ids if m not in user_rated_movies.values]
    predictions = [model.predict(user_id, movie_id) for movie_id in unrated_movie_ids]
    predictions.sort(key=lambda x: x.est, reverse=True)  # Sort by estimated rating
    top_recommendations = predictions[:n]
    recommended_movie_ids = [int(pred.iid) for pred in top_recommendations]
    recommended_movies = movies_df[movies_df['movieId'].isin(recommended_movie_ids)]
    return recommended_movies[['movieId', 'title']]
user_id = 1
recommendations = recommend_movies(user_id, svd_model, movies_df, n=10)
print("Top 10 recommended movies:")
print(recommendations)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8696  0.8775  0.8655  0.8805  0.8700  0.8726  0.0055  
MAE (testset)     0.6685  0.6735  0.6642  0.6757  0.6679  0.6700  0.0041  
Fit time          2.14    1.51    1.49    2.31    1.52    1.79    0.36    
Test time         0.12    0.27    0.12    0.34    0.12    0.19    0.09    
Top 10 recommended movies:
     movieId                                              title
210      246                                 Hoop Dreams (1994)
277      318                   Shawshank Redemption, The (1994)
413      475                   In the Name of the Father (1993)
474      541                                Blade Runner (1982)
602      750  Dr. Strangelove or: How I Learned to Stop Worr...
659      858                              Godfather, The (1972)
684      902                      Breakfast at Tiffany's (1961)
690      908                    